<div class="alert alert-success">  
</div>

<div> 
    <h2 align="center">Tabular Playground Series - Oct 2021</h2> 
    <h1 align="center">LightGBM & AUC Evaluation [ROC curve]</h1>            
    <h4 align="center">By: Somayyeh Gholami & Mehran Kazeminia</h4>
</div>

<div class="alert alert-success">  
</div>

#### Hello everyone,

#### In this notebook, we examine the following questions and try to find the answers to these questions.

#### **1- Is it possible to ignore some "folds" in the "cross-validation" method? And is it possible that this will cause our score to increase?**

#### Sometimes local maxima or local minima lock our calculations, or for any other reason, our algorithm for one or more "folds" is not well trained. If we ignore these "folds", we may be able to get a better score in the prediction phase for test data.

#### In this notebook, we defined ten "folds" for "cross-validation" and considered all "folds" for the first time. The second time, we excluded five "folds" that had less "accuracy". This improved our score in the forecast phase for test data. Of course, this may not always be true. However, it is conceivable that perhaps this simple task in some challenges can easily improve our score.

#### **2- Is it possible to use all the training data first and then use it as a "fold" in "cross-validation" when teaching the algorithm? And can this work improve our score?**

#### In this notebook, we did exactly that with "Ensembling" and got a better score. However, you should always try, and this may not always improve your score.

#### Please note that you will need to make some adjustments again. For example, if you are using LightGBM, you will need to set the value of n_estimators again to use all the training data. Because "valid_X" and "valid_y" no longer exist. However, we hope our code is easy to understand.

#### **3- Can we train the algorithm separately using "Categorical Features" and "Numerical Features"? And then "Ensemble" the results? And is it possible that this will improve our score?**

#### We did this at the end of this notebook with the titles "test3", "test4" and "test5". The form of the histogram curves of the results is very interesting and follows the mathematical logic of this topic. It's an interesting experience, but don't expect it to improve your score. Because it does not help the decisions of the algorithm and reduces its certainty.

#### **4- What are the weaknesses of AUC Evaluation?**

#### We have addressed this issue at the beginning of the notebook as well as at the end of the notebook under the titles "test1" and "test2".

#### Please note that we can not change the ranking of some results or change the value of some results without a reason, even if "AUC Evaluation" is not sensitive to these shifts and changes. This means that, as you can see in this notebook, "AUC Evaluation" may give a good score to completely wrong results.

### **Good Luck.**


<div class="alert alert-success">
    <h3 align="center">If you find this work useful, please don't forget upvoting :)</h3>
</div>

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px

%matplotlib inline

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc

from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import minmax_scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif

In [ ]:
from lightgbm import LGBMClassifier

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Competition Evaluation</span>**

#### Submissions are evaluated on area under the **ROC curve** between the predicted probability and the observed target.

<img src="https://raw.githubusercontent.com/MehranKazeminia/fifa-worldcup-2018/master/roc403.jpg">

In [ ]:
def roc_auc(true_list, pred_list, a, b):
    
    fpr, tpr, _ = roc_curve(true_list, pred_list)    
    roc_auc = auc(fpr, tpr)

    print(f'FPR: {fpr}')
    print(f'TPR: {tpr}')
    #print(f'{list(zip(fpr,tpr))}') 
    print(f'\n>>>>> ROC_AUC: %0.6f <<<<<' %roc_auc)
    
    plt.style.use('seaborn-whitegrid')
    plt.figure(figsize=(a, b), facecolor='lightgray')
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.01, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'\nThe area under the ROC curve\n')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
true_list  = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

pred_list1 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# The results are quite similar:
# pred_list1 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
# pred_list1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# pred_list1 = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])
# pred_list1 = np.array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3])
# pred_list1 = np.array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500])
# pred_list1 = np.array([-50, -50, -50, -50, -50, -50, -50, -50, -50, -50])
# pred_list1 = np.array([500, -50, 500, -50, 500, -50, 500, -50, 500, 500])
# This means that this type of evaluation does not have the ability to detect some errors.

roc_auc(true_list, pred_list1, 7, 7)

In [ ]:
true_list  = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

pred_list2 = np.array([1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# The results are quite similar:
# pred_list2 = np.array([0.6, 0.8, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
# pred_list2 = np.array([0.8, 0.6, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
# pred_list2 = np.array([120, 850, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) 
# pred_list2 = np.array([850, 120, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) 
# This type of evaluation does not have the ability to detect changes in some rankings.

roc_auc(true_list, pred_list2, 7, 7)

In [ ]:
true_list  = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

pred_list3 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0])

# The results are quite similar:
# pred_list3 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.2, 0.4])
# pred_list3 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.4, 0.2])
# pred_list3 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, -25, -85])
# pred_list3 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, -85, -25])
# This type of evaluation does not have the ability to detect changes in some rankings.

roc_auc(true_list, pred_list3, 7, 7)

In [ ]:
true_list  = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

pred_list4 = np.array([0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

# The results are quite similar:
# pred_list4 = np.array([0.2, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
# pred_list4 = np.array([0.4, 0.2, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
# pred_list4 = np.array([-35, -65, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) 
# pred_list4 = np.array([-65, -35, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) 
# This type of evaluation does not have the ability to detect changes in some rankings.

roc_auc(true_list, pred_list4, 7, 7)

In [ ]:
true_list  = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

pred_list5 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0])

# The results are quite similar:
# pred_list5 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.8])
# pred_list5 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.8, 0.6])
# pred_list5 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 150, 670])
# pred_list5 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 670, 150])
# This type of evaluation does not have the ability to detect changes in some rankings.

roc_auc(true_list, pred_list5, 7, 7)

### For more information, refer to the following address:

### [Learning from imbalanced data.](http://www.jeremyjordan.me/imbalanced-data/)

![](https://www.jeremyjordan.me/content/images/2018/11/roc_cutoff-1.gif)

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Challenge Datasets</span>**


In [ ]:
import datatable as dt 

DF1 = dt.fread('../input/tabular-playground-series-oct-2021/train.csv').to_pandas()

DF2 = dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas()

SAM = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()

display(DF1.shape, DF2.shape, SAM.shape)

In [ ]:
# Check Null Values
MV1 = DF1.isnull().sum()
MV2 = DF2.isnull().sum()

print(f'Missing Value DF1:\n{MV1[MV1 > 0]}\n')
print(f'Missing Value DF2:\n{MV2[MV2 > 0]}\n')

In [ ]:
display(DF1, DF2)

# display(DF1.describe().transpose())
# display(DF2.describe().transpose())

In [ ]:
print('=' * 40)
DF1.info(memory_usage='deep')
print('=' * 40)
DF2.info(memory_usage='deep')
print('=' * 40)

In [ ]:
DF1['target'].value_counts().plot(figsize=(4, 4), kind='bar')

In [ ]:
DF1['target'].value_counts().plot(figsize=(6, 6), kind='pie')

DF1['target'].value_counts(normalize=True)

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Compressing</span>**

#### Thanks to: **@gemartin** & **@lucamassaron**

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
DF1 = reduce_memory_usage(DF1)

DF2 = reduce_memory_usage(DF2)

### Prepare datasets for modeling

In [ ]:
X  = DF1.drop(columns = ['id','target'])

XX = DF2.drop(columns = ['id'])

y  = DF1.target

display(X, XX, y)

In [ ]:
import gc
del DF1
del DF2
gc.collect()

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Features</span>**

In [ ]:
features = X.columns

print(features)
print(f'\n>>>>> Number: {len(features)} <<<<<') 

### Categorical Features & Numerical Features

In [ ]:
categ_featu = [col for col in X.columns if X[col].dtype == 'bool']

numer_featu = [col for col in X.columns if X[col].dtype != 'bool']

print(categ_featu)
print(f'\n>>>>> Number: {len(categ_featu)} <<<<<\n')

print(numer_featu)
print(f'\n>>>>> Number: {len(numer_featu)} <<<<<\n')

### Data Augmentation

In [ ]:
X['categ_sum'] = X[categ_featu].sum(axis=1)

XX['categ_sum'] = XX[categ_featu].sum(axis=1)
                                        
numer_featu += ['categ_sum'] 

print(numer_featu)
print(f'\n>>>>> Number: {len(numer_featu)} <<<<<\n')

In [ ]:
hist_data = [XX['categ_sum'], X['categ_sum']]  
group_labels = ['XX_categ_sum', 'X_categ_sum']
    
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show()

In [ ]:
X[numer_featu].dtypes

In [ ]:
X[categ_featu].dtypes

In [ ]:
# for c in categ_featu:    
#    X[c] = X[c].astype('category')  
#    XX[c] = XX[c].astype('category') 

# X[categ_featu].dtypes

In [ ]:
categorical_mask = ([False] * 22  +
                    [True]  * 1   +
                    [False] * 20  +
                    [True]  * 1   +
                    [False] * 198 +
                    [True]  * 43  +
                    [False] * 1   )

print(categorical_mask)
print(f'\n>>>>> Number: {len(categorical_mask)} <<<<<\n')

In [ ]:
# display(X, XX)

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">LGBMClassifier</span>** 

## Model (Cross Validation)

In [ ]:
model = LGBMClassifier(n_estimators= 15000, 
                       learning_rate= 0.008, 
                       objective= 'binary',                      
                       metric= 'auc',                       
                       reg_alpha= 10,
                       reg_lambda= 0.1,                     
                       num_leaves= 31,
                       max_depth= -1,
                       subsample= 0.6,
                       subsample_freq= 1, 
                       colsample_bytree= 0.4,
                       min_child_weight= 256,
                       min_child_samples= 20, 
                       random_state= 123)

In [ ]:
N = 0  # Counter
F = 10 # Number of Splits
pred = np.zeros(XX.shape[0])
pred_select = np.zeros(XX.shape[0])

skf = StratifiedKFold(n_splits= F, shuffle= True, random_state= 123)

In [ ]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    
    print(f':::::::::::::: f o l d >>> {fold} :::::::::::::::\n')
    
    train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
    valid_X, valid_y = X.iloc[valid_idx], y.iloc[valid_idx]
    
    model.fit(train_X, train_y,
              eval_metric= 'auc',
              eval_set= [(valid_X, valid_y)], 
              early_stopping_rounds= 200,
              verbose= 500)
    
    oof_predict = model.predict(valid_X)
    accuracy = accuracy_score(valid_y, oof_predict)   
    print(f'\nPrediction: {oof_predict}\n')
    print(f'Accuracy: {accuracy}\n')
    roc_auc(valid_y, oof_predict, 7, 7)
   
    
    print(f':::::::::: Main Results - fold {fold} ::::::::::\n')
    oof_pred = model.predict_proba(valid_X)[:, -1]
    print(f'Prediction: {oof_pred}\n')
    roc_auc(valid_y, oof_pred, 7, 7)
    
    # hist_data = [oof_pred, oof_predict, valid_y]  
    # group_labels = ['Predict_proba', 'Predict', 'Correct Value']  
    # fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
    # fig.show()                                             

    pred += model.predict_proba(XX)[:, -1]
    
    if (accuracy > 0.7696):  
        pred_select += model.predict_proba(XX)[:, -1]
        N = N + 1
        
    if (fold == 0):
        train_X_test = train_X.copy()
        np.save('train_X_test.npy', train_X_test)
        
        train_y_test = train_y.copy()
        np.save('train_y_test.npy', train_y_test)
        
        valid_X_test = valid_X.copy()
        np.save('valid_X_test.npy', valid_X_test)
        
        valid_y_test = valid_y.copy()
        np.save('valid_y_test.npy', valid_y_test)
        
        oof_pred_test = oof_pred.copy()
        np.save('oof_pred_test.npy', oof_pred_test)
        
        oof_predict_test = oof_predict.copy()
        np.save('oof_predict_test.npy', oof_predict_test)
        
                                   
print(f'::::::::::::::: F I N A L  R E S U L T ::::::::::::::::\n')

pred = pred / F
pred_select = pred_select / N

print(f'Number of Total Folds: {F}')
print(f'Number of Selected Folds: {N}')
print('=' * 70, '\n')

print('Based on Total Folds:')
print(f'Min: {pred.min()}, Max: {pred.max()}\n')
pred = np.clip(pred, y.min(), y.max())
np.save('pred.npy', pred)
print(pred, pred.shape)
print('=' * 70, '\n')

print('Based on Selected Folds:')
print(f'Min: {pred_select.min()}, Max: {pred_select.max()}\n')
pred_select = np.clip(pred_select, y.min(), y.max())
np.save('pred_select.npy', pred_select)
print(pred_select, pred_select.shape)
print('=' * 70, '\n')

hist_data = [pred_select, pred]  
group_labels = ['Based on Selected Folds', 'Based on Total Folds']    
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show() 

gc.collect()  

In [ ]:
sub1 = SAM.copy()

sub1['target'] = pred
display(sub1)

In [ ]:
sub2_select = SAM.copy()

sub2_select['target'] = pred_select
display(sub2_select)

In [ ]:
sub1.to_csv("submission1.csv",index=False)
sub2_select.to_csv("submission2_select.csv",index=False)
!ls

<div class="alert alert-success">  
</div>

## Model (For all Data)

In [ ]:
modelt= LGBMClassifier(n_estimators= 8000, 
                       learning_rate= 0.008, 
                       objective= 'binary',                      
                       metric= 'auc',                       
                       reg_alpha= 10,
                       reg_lambda= 0.1,                     
                       num_leaves= 31,
                       max_depth= -1,
                       subsample= 0.6,
                       subsample_freq= 1, 
                       colsample_bytree= 0.4,
                       min_child_weight= 256,
                       min_child_samples= 20, 
                       random_state= 123)

In [ ]:
modelt.fit(X, y, eval_metric= 'auc', verbose= 200)

In [ ]:
predt = modelt.predict_proba(XX)[:, -1]

print(f'Min: {predt.min()}, Max: {predt.max()}\n')  
predt = np.clip(predt, y.min(), y.max())
print(predt, predt.shape)

In [ ]:
hist_data = [predt]  
group_labels = ['Predicted (For all Data)']
    
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show()

In [ ]:
sub3_all_data = SAM.copy()

sub3_all_data['target'] = predt
display(sub3_all_data)

In [ ]:
sub3_all_data.to_csv("submission3_all_data.csv",index=False)
!ls

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Ensembling</span>** 


In [ ]:
pred_ense_a = (pred * 0.80) + (predt * 0.20)

print(pred_ense_a, pred_ense_a.shape)

In [ ]:
pred_ense_b = (pred_select * 0.80) + (predt * 0.20)

print(pred_ense_b, pred_ense_b.shape)

In [ ]:
sub4_ense_a = SAM.copy()

sub4_ense_a['target'] = pred_ense_a
display(sub4_ense_a)

In [ ]:
sub5_ense_b = SAM.copy()

sub5_ense_b['target'] = pred_ense_b
display(sub5_ense_b)

In [ ]:
sub4_ense_a.to_csv("submission4_ense_a.csv",index=False)
sub5_ense_b.to_csv("submission5_ense_b.csv",index=False)
!ls

<div>
    <h1 align="center"> >>> End of Submissions<<< </h1>
</div>

<div class="alert alert-success">  
</div>

#### submission1.csv >>>>>>>>>> Public Score >>>> 0.85633 >>>> The classic "cross-validation" method

#### submission2_select.csv >>>>> Public Score >>>> 0.85634 >>>> Remove bad folds

#### submission3_all_data.csv >>>> Public Score >>>> 0.85625 >>>> Use all data

#### submission4_ense_a.csv >>>>> Public Score >>>> 0.85634 >>>> Blend the first with the third

#### submission5_ense_b.csv >>>>> Public Score >>>> 0.85635 >>>> Blend the second with the third

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Another example of poor evaluation</span>** 

## Test: 1 >>> Results to the power of four

In [ ]:
oof_pred_test1 = oof_pred_test.copy()

oof_pred_test1

In [ ]:
for i in range(len(oof_pred_test1)):
    
    oof_pred_test1[i] = oof_pred_test1[i] ** 4
                
oof_pred_test1        

In [ ]:
hist_data = [oof_pred_test1, oof_pred_test]  
group_labels = ['Results to the power of four', 'Predicted Results']
    
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show()

### But the evaluation for the main results and the evaluation for the results to the power of four are exactly equal.

In [ ]:
# Before the changes
roc_auc(valid_y_test, oof_pred_test, 7, 7)

In [ ]:
# After the changes
roc_auc(valid_y_test, oof_pred_test1, 7, 7)

In [ ]:
print(oof_pred_test1.min(), oof_pred_test1.max())

<div class="alert alert-success">  
</div>

## Test: 2 >>> Results Changed

In [ ]:
oof_pred_test2 = oof_pred_test.copy()

oof_pred_test2

In [ ]:
for i in range(len(oof_pred_test2)):
    
    if (oof_pred_test2[i] < 0.5):        
        oof_pred_test2[i] = oof_pred_test1[i] ** 1.2
        
    if (oof_pred_test2[i] > 0.5):        
        oof_pred_test2[i] = oof_pred_test1[i] ** 0.8
        
oof_pred_test2

In [ ]:
hist_data = [oof_pred_test2, oof_pred_test]  
group_labels = ['Results Changed', 'Predicted Results']
    
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show()

### But evaluating the main results and evaluating the changed results are still the same.

In [ ]:
# Before the changes
roc_auc(valid_y_test, oof_pred_test, 7, 7)

In [ ]:
# After the changes
roc_auc(valid_y_test, oof_pred_test2, 7, 7)

In [ ]:
print(oof_pred_test2.min(), oof_pred_test2.max())

<div class="alert alert-success">  
</div>

# **<span style="color:darkgray;">Use Categorical Features and Numerical Features separately</span>** 

## Test: 3 >>> Only Categorical Features

In [ ]:
X[categ_featu]

In [ ]:
model.fit(train_X_test[categ_featu], train_y_test,     
          eval_metric= 'auc',
          eval_set= [(valid_X_test[categ_featu], valid_y_test)], 
          early_stopping_rounds= 200,
          verbose= 200)   

In [ ]:
oof_predict_test3 = model.predict(valid_X_test[categ_featu])

accuracy = accuracy_score(valid_y_test, oof_predict_test3) 

print(f'\nPrediction: {oof_predict_test3}\n')

print(f'Accuracy: {accuracy}\n')

roc_auc(valid_y_test, oof_predict_test3, 7, 7)

In [ ]:
oof_pred_test3 = model.predict_proba(valid_X_test[categ_featu])[:, -1]

print(f'\nPrediction: {oof_pred_test3}\n')

roc_auc(valid_y_test, oof_pred_test3, 7, 7)   

In [ ]:
hist_data = [oof_pred_test3, oof_predict_test3, valid_y_test] 
group_labels = ['Predict_proba', 'Predict', 'Correct Value']  

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show() 

<div class="alert alert-success">  
</div>

## Test: 4 >>> Only Numerical Features

In [ ]:
X[numer_featu]

In [ ]:
model.fit(train_X_test[numer_featu], train_y_test,     
          eval_metric= 'auc',
          eval_set= [(valid_X_test[numer_featu], valid_y_test)], 
          early_stopping_rounds= 200,
          verbose= 200) 

In [ ]:
oof_predict_test4 = model.predict(valid_X_test[numer_featu])

accuracy = accuracy_score(valid_y_test, oof_predict_test4) 

print(f'\nPrediction: {oof_predict_test4}\n')

print(f'Accuracy: {accuracy}\n')

roc_auc(valid_y_test, oof_predict_test4, 7, 7)

In [ ]:
oof_pred_test4 = model.predict_proba(valid_X_test[numer_featu])[:, -1]

print(f'\nPrediction: {oof_pred_test4}\n')

roc_auc(valid_y_test, oof_pred_test4, 7, 7)   

In [ ]:
hist_data = [oof_pred_test4, oof_predict_test4, valid_y_test] 
group_labels = ['Predict_proba', 'Predict', 'Correct Value'] 

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show() 

<div class="alert alert-success">  
</div>

## Test: 5 >>> Only Categorical Features + Only Numerical Features

In [ ]:
oof_pred_test5 = (oof_pred_test3 * 0.5) + (oof_pred_test4 * 0.5)

print(f'\nPrediction: {oof_pred_test5}\n')

roc_auc(valid_y_test, oof_pred_test5, 7, 7)  

In [ ]:
hist_data = [oof_pred_test, oof_pred_test3, oof_pred_test4, oof_pred_test5]  
group_labels = ['Main Result', 'Categorical', 'Numerical', 'Categorical + Numerical']  

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False) 
fig.show()

<div class="alert alert-success">  
</div>

<div class="alert alert-success">  
</div>